In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
import seaborn as sns
from sklearn.model_selection import cross_val_predict


# Meal Speech vs Meal Movement

In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
play_data = play_data.dropna()

In [ ]:
## Just movement

# Perform a grid search for each classifier
X = meal_data[[
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max']]
y = meal_data['label'].iloc[:, 0]



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf', probability=True), {
        'pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

In [ ]:
model_movement = best_models['svc_rbf']['best_estimator']

In [ ]:
meal_data.columns

In [ ]:
## Just speech

# Perform a grid search for each classifier
X = play_data[['Proportion speech child']]
y = play_data['label'].iloc[:, 0]



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear', probability=True), {
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf', probability=True), {
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                            (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
X

In [ ]:
model_speech = best_models['svc_rbf']['best_estimator']

## Get the subset samples from both meal and play 

In [ ]:
meal_data_selected = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group','label']]
play_data_selected = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group', 'label']]

# Find the common indices
common_indices = meal_data_selected.index.intersection(play_data_selected.index)

# Filter the DataFrames to keep only the common indices
meal_data_filtered = meal_data_selected.loc[common_indices]
play_data_filtered = play_data_selected.loc[common_indices]

# Concatenate the filtered DataFrames
combined_data = pd.concat([meal_data_filtered, play_data_filtered], axis=1)
combined_data = combined_data.dropna()

In [ ]:
len(meal_data_filtered), len(play_data_filtered)

In [ ]:
## correlations between the two models 
# Get predictions from the LR model
movement_features = meal_data_filtered[[
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max']]
speech_features = play_data_filtered[[
         'Proportion speech child',
]]


In [ ]:
len(speech_features), len(movement_features), len(combined_data)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
predictions_speech = cross_val_predict(model_speech, speech_features, combined_data['label'].iloc[:,0], cv=cv)
predictions_movement = cross_val_predict(model_movement, movement_features, combined_data['label'].iloc[:,0], cv=cv)

In [ ]:
agreements = np.sum(predictions_speech == predictions_movement)

# Calculate the percentage of agreements
percentage_agreement = (agreements / len(predictions_speech)) * 100

print(f'Percentage of agreement: {percentage_agreement:.2f}%')

In [ ]:
from scipy.stats import pearsonr, binomtest
# Perform a binomial test to determine if the agreement rate is significantly higher than chance
# The null hypothesis is that the agreement rate is 50% (random chance)
p_value_agreement = binomtest(agreements, len(predictions_speech), p=0.5, alternative='greater')

# Print the agreement rate and the p-value
print(f'P-value for agreement rate being higher than chance: {p_value_agreement}')

In [ ]:
p_values = []
for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    predictions_speech = cross_val_predict(model_speech, speech_features, combined_data['label'].iloc[:,0], cv=cv)
    predictions_movement = cross_val_predict(model_movement, movement_features, combined_data['label'].iloc[:,0], cv=cv)
    agreements = np.sum(predictions_speech == predictions_movement)
    p_value_agreement = binomtest(agreements, len(predictions_speech), p=0.5, alternative='greater').pvalue
    p_values.append(p_value_agreement)

In [ ]:
print(f'Mean p-value: {np.mean(p_values)}') 

# 	Comparing misclassifications between the models.

In [ ]:
# True labels
true_labels = combined_data['label'].iloc[:,0]

# Identify misclassifications
misclassifications_speech = predictions_speech != true_labels
misclassifications_movement = predictions_movement != true_labels

# Compare misclassifications
speech_correct_movement_wrong = (predictions_speech == true_labels) & (predictions_movement != true_labels)
movement_correct_speech_wrong = (predictions_movement == true_labels) & (predictions_speech != true_labels)

# Calculate proportions
total_samples = len(true_labels)
proportion_speech_correct_movement_wrong = np.sum(speech_correct_movement_wrong) / total_samples
proportion_movement_correct_speech_wrong = np.sum(movement_correct_speech_wrong) / total_samples

# Print results 
print(f'Proportion of cases where speech model is correct but movement model is wrong: {proportion_speech_correct_movement_wrong:.2%}')
print(f'Proportion of cases where movement model is correct but speech model is wrong: {proportion_movement_correct_speech_wrong:.2%}')

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
contingency_table = np.array([
    [np.sum((predictions_speech == true_labels) & (predictions_movement == true_labels)), np.sum(speech_correct_movement_wrong)],
    [np.sum(movement_correct_speech_wrong), np.sum((predictions_speech != true_labels) & (predictions_movement != true_labels))]
])

# Perform McNemar's test
result = mcnemar(contingency_table, exact=True)

print(f'McNemar\'s test statistic: {result.statistic}')
print(f'P-value: {result.pvalue}')

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)

predictions_speech_proba = cross_val_predict(model_speech, speech_features, combined_data['label'].iloc[:,0], cv=cv, method='predict_proba')
predictions_movement_proba = cross_val_predict(model_movement, movement_features, combined_data['label'].iloc[:,0], cv=cv, method='predict_proba')


In [ ]:
from scipy.stats import pearsonr
# Calculate the Pearson correlation coefficient
predictions_speech = predictions_speech_proba[:, 1]
predictions_movement = predictions_movement_proba[:, 1]

correlation, p_value = pearsonr(predictions_speech, predictions_movement)

# Print the results
print(f'Pearson correlation coefficient: {correlation:.2f}')
print(f'P-value: {p_value}')